In [2]:
library(data.table)
library(arrow)

Some features are not enabled in this build of Arrow. Run `arrow_info()` for more information.


Attaching package: 'arrow'


The following object is masked from 'package:utils':

    timestamp




In [3]:
################# COMPILATION #################

# database <- do.call(rbind, list_files)
# database <- as.data.frame(database)

pre_assembled_file = 'Input/Processed/osha_data.csv'
database <- fread(pre_assembled_file, na.strings=c("", "nan"))

################# RE-FORMAT #################

## Re adjusting type and format of variables

lct <- Sys.getlocale("LC_TIME"); Sys.setlocale("LC_TIME", "C") #format error for date without this command

database <- within(database, {
  
  AIR_VOLUME_SAMPLED <- as.numeric(as.character(AIR_VOLUME_SAMPLED))
  
  BLANK_USED <- factor(BLANK_USED, levels = c('Y', 'N'))
  
  CITY <- as.character(CITY)
  
  DATE_REPORTED <- as.character(tolower(DATE_REPORTED))
  DATE_REPORTED <- ifelse(nchar(DATE_REPORTED) == 11 , as.character(as.Date(DATE_REPORTED, format = "%Y-%b-%d")), 
                          ifelse(nchar(DATE_REPORTED) == 19, as.character(as.Date(DATE_REPORTED, format = '%Y/%m/%d')), 
                                 as.character(as.Date(DATE_REPORTED, format = '%Y-%m-%d'))))
  DATE_REPORTED <- as.Date(DATE_REPORTED, format = '%Y-%m-%d')
  
  DATE_SAMPLED <- as.character(tolower(DATE_SAMPLED))
  DATE_SAMPLED <- ifelse(nchar(DATE_SAMPLED) == 11 , as.character(as.Date(DATE_SAMPLED, format = "%Y-%b-%d")), 
                         ifelse(nchar(DATE_SAMPLED) == 19, as.character(as.Date(DATE_SAMPLED, format = '%Y/%m/%d')), 
                                as.character(as.Date(DATE_SAMPLED, format = '%Y-%m-%d'))))
  DATE_SAMPLED <- as.Date(DATE_SAMPLED, format = '%Y-%m-%d')
  
  EIGHT_HOUR_TWA_CALC <- factor(EIGHT_HOUR_TWA_CALC, levels = c('Y', 'N'))
  
  ESTABLISHMENT_NAME <- as.character(ESTABLISHMENT_NAME)
  
  FIELD_NUMBER <- as.character(FIELD_NUMBER)
  
  IMIS_SUBSTANCE_CODE <- factor(gsub(" ", "0", sprintf("%4s", database$IMIS_SUBSTANCE_CODE)))
  INSPECTION_NUMBER <- factor(INSPECTION_NUMBER)
  
  INSTRUMENT_TYPE <- as.character(INSTRUMENT_TYPE) #need to be cleaned before levels
  
  LAB_NUMBER <- factor(LAB_NUMBER)
  
  NAICS_CODE <- as.character(NAICS_CODE)
  NAICS_CODE <- ifelse(nchar(NAICS_CODE) < 6, NA, NAICS_CODE) #deletion of missing or 0 NAICS CODE
  
  OFFICE_ID <- factor(OFFICE_ID)
  
  QUALIFIER <- as.character(QUALIFIER) #need to be cleaned before levels
  
  SAMPLE_RESULT <- as.numeric(as.character(database$SAMPLE_RESULT))
  
  ## Should have only 4 levels according to CEHD information
  SAMPLE_TYPE <- factor(SAMPLE_TYPE) #A B BL BU L M P S U W WB Z
  
  SAMPLE_WEIGHT <- as.numeric(as.character(SAMPLE_WEIGHT))
  
  ## Maybe need a cleaning. Kept it as factor for now on.
  SAMPLING_NUMBER <- factor(SAMPLING_NUMBER)
  
  SIC_CODE <- factor(SIC_CODE)
  
  STATE <- factor(STATE)
  
  # Label of IMIS code, not the same number of levels though.
  SUBSTANCE <- as.character(SUBSTANCE)
  
  # Time in minutes
  TIME_SAMPLED <- as.numeric(as.character(TIME_SAMPLED))
  
  UNIT_OF_MEASUREMENT <- as.character(UNIT_OF_MEASUREMENT) # #need to be cleaned before levels
  
  ZIP_CODE <- factor(gsub(" ", "0", sprintf("%5s", database$ZIP_CODE)))
  
})


database$YEAR <- factor(substring(database$DATE_SAMPLED, 1, 4))

[1] "C"

Warning message in eval(substitute(expr), e):
"NAs introduced by coercion"


In [4]:
##################################################################/#


# ##################################################################/#
# ############################ CLEANING #############################
# ##################################################################/#
# 
#load db
# database <- readRDS("C:\\Users\\phisar\\Dropbox (IRSST)\\PhD\\Projet IMIS\\R�sultats\\IMIS_ND_predict\\CEHD 84_18\\database_all_CEHD.rds")
dim(database) #[1] 2395071      26
table(database$YEAR)
names(database)
#[1] "AIR_VOLUME_SAMPLED"  "BLANK_USED"          "CITY"                "DATE_REPORTED"       "DATE_SAMPLED"       
#[6] "EIGHT_HOUR_TWA_CALC" "ESTABLISHMENT_NAME"  "FIELD_NUMBER"        "IMIS_SUBSTANCE_CODE" "INSPECTION_NUMBER"  
#[11] "INSTRUMENT_TYPE"     "LAB_NUMBER"          "NAICS_CODE"          "OFFICE_ID"           "QUALIFIER"          
#[16] "SAMPLE_RESULT"       "SAMPLE_TYPE"         "SAMPLE_WEIGHT"       "SAMPLING_NUMBER"     "SIC_CODE"           
#[21] "STATE"               "SUBSTANCE"           "TIME_SAMPLED"        "UNIT_OF_MEASUREMENT" "ZIP_CODE"           
#[26] "YEAR"           
database$IMIS_SUBSTANCE_CODE <- as.character(database$IMIS_SUBSTANCE_CODE)
#class(database$IMIS_SUBSTANCE_CODE)
#table(database$IMIS_SUBSTANCE_CODE)
#sort(unique(database$IMIS_SUBSTANCE_CODE))
length(unique(database$IMIS_SUBSTANCE_CODE)) #[1] 1159
str(database)
database$INSPECTION_NUMBER <- as.character(database$INSPECTION_NUMBER)
database$SAMPLING_NUMBER <- as.character(database$SAMPLING_NUMBER)

#remove white spaces left and right of variables
database$INSPECTION_NUMBER <- trimws(database$INSPECTION_NUMBER)
database$SAMPLING_NUMBER <- trimws(database$SAMPLING_NUMBER)

### Setting data.frame to count elimination
reasons <- data.frame(YEAR = min(as.character(database$YEAR)) : max(as.character(database$YEAR)))

Your code contains a unicode char which cannot be displayed in your
current locale and R will silently convert it to an escaped form when the
R kernel executes this code. This can lead to subtle errors if you use
such chars to do comparisons. For more information, please see
https://github.com/IRkernel/repr/wiki/Problems-with-unicode-on-windows

[1] 2395071      27


  1984   1985   1986   1987   1988   1989   1990   1991   1992   1993   1994 
 80878 119458 108997 126963 139199 104360 117812  66581  49366  79072  73987 
  1995   1996   1997   1998   1999   2000   2001   2002   2003   2004   2005 
 47501  47971  59143  54053  62721  53764  54597  72513  56616  65005  60389 
  2006   2007   2008   2009   2010   2011   2013   2014   2015   2016   2017 
 57880  50674  47778  51095  64219  73494  63718  65741  58971  63505  54052 
  2018 
 42998 

[1] "V1"                  "AIR_VOLUME_SAMPLED"  "BLANK_USED"         
 [4] "CITY"                "DATE_REPORTED"       "DATE_SAMPLED"       
 [7] "EIGHT_HOUR_TWA_CALC" "ESTABLISHMENT_NAME"  "FIELD_NUMBER"       
[10] "IMIS_SUBSTANCE_CODE" "INSPECTION_NUMBER"   "INSTRUMENT_TYPE"    
[13] "LAB_NUMBER"          "NAICS_CODE"          "OFFICE_ID"          
[16] "QUALIFIER"           "SAMPLE_RESULT"       "SAMPLE_TYPE"        
[19] "SAMPLE_WEIGHT"       "SAMPLING_NUMBER"     "SIC_CODE"           
[22] "STATE"               "SUBSTANCE"           "TIME_SAMPLED"       
[25] "UNIT_OF_MEASUREMENT" "YEAR"                "ZIP_CODE"

[1] 1159

Classes 'data.table' and 'data.frame':	2395071 obs. of  27 variables:
 $ V1                 : int  0 1 2 3 4 5 6 7 8 9 ...
 $ AIR_VOLUME_SAMPLED : num  516 NA 76 NA 74 0 0 74 0 74 ...
 $ BLANK_USED         : Factor w/ 2 levels "Y","N": 2 1 2 1 2 1 1 2 1 2 ...
 $ CITY               : chr  "Plainview" "Plainview" "Plainview" "Plainview" ...
 $ DATE_REPORTED      : Date, format: "1984-06-15" "1984-06-15" ...
 $ DATE_SAMPLED       : Date, format: "1984-05-07" "1984-05-07" ...
 $ EIGHT_HOUR_TWA_CALC: Factor w/ 2 levels "Y","N": NA NA NA NA NA NA NA NA NA NA ...
 $ ESTABLISHMENT_NAME : chr  "CONTROLLED CASTINGS CORP" "CONTROLLED CASTINGS CORP" "CONTROLLED CASTINGS CORP" "CONTROLLED CASTINGS CORP" ...
 $ FIELD_NUMBER       : chr  "2LI8491" "2LI8492" "2LI8493" "2LI8494" ...
 $ IMIS_SUBSTANCE_CODE: chr  "T100" "T100" "T100" "T100" ...
 $ INSPECTION_NUMBER  : Factor w/ 91006 levels "019-17","100000231",..: 27240 27240 27240 27240 27255 27255 27255 27255 27255 27255 ...
 $ INSTRUMENT_TYPE    : ch

In [5]:
# ##################### N01: Blanks ####
# N01 removing blanks from the blank_used variable (other blanks identified later by qualifier)

vec_eff <- table(database$YEAR[database$BLANK_USED != 'N'])
reasons$N01[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[database$BLANK_USED == 'N', ]
table(database$BLANK_USED)
#Y       N 
#0 1958557
sum(reasons$N01,na.rm = TRUE)
length(database[,1]) #1958557(-436514)


      Y       N 
      0 1958557 

[1] 436514

[1] 1

In [6]:
# ##################### N02: Personal measurements ####
# N02 removing non personal samples

vec_eff <- table(database$YEAR[database$SAMPLE_TYPE != 'P'])
reasons$N02[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[database$SAMPLE_TYPE == 'P', ] #excluding all things that are not P (keeping P only)
table(database$SAMPLE_TYPE)
#A       B      BL      BU       L       M       P       S       U       W      WB       Z 
#0       0       0       0       0       0 1506202       0       0       0       0       0
sum(reasons$N02,na.rm = TRUE)
length(database[,1]) #1506271(-452286)


      A       B      BL      BU       L       M    None       P       S       U 
      0       0       0       0       0       0       0 1506202       0       0 
      W      WB       Z 
      0       0       0 

[1] 452355

[1] 1

In [7]:
# ##################### N03 Excluding substances with few samples ####
# Store the 81 substances of the 2011 initial cleaning procedure in an object
sub_list_81 <-c('0040','0230','0260','0360','0430','0491','0685','0720','0731','1073','1290','1520','1560',
           '1591','1620','1730','1790','1840','2270','2280','2460','2571','2590','2610','9010','9020','9130','9135','C141',
           '0170','0320','0435','0440','0460','0490','0686','0689','0692','0700','0710','0726','0730','0830','0860','1037','1040',
           '1060','1080','1190','1280','1371','1377','1380','1385','1430','1534','1536','1631','1660','1675','1720','1774','1860','2020',
           '2037','2040','2170','2180','2240','2260','2290','2310','2430','2470','2490','2505','2580','9015','T177','V109','S103')
sub_list_81 <- sort(sub_list_81)

# Store all substances with >100 records in an object
subst <- data.frame(table(database$IMIS_SUBSTANCE_CODE))
names(subst) <-c('code','n')
subst <- subst[order(-subst$n),]
nrow(subst) #[1] 978
subst$name <- database$SUBSTANCE[match(subst$code,database$IMIS_SUBSTANCE_CODE)]
subst <- subst[subst$n>=100,]
nrow(subst) #[1] 265
#View(subst)
# remove codes which do not correspond to chemical substances
subst_all <- subst[!is.element(subst$code,c('G301','G302','Q115','T110','M125','Q116','Q100','S325')),]
nrow(subst_all) #[1] 257
sub_list_all <- sort(as.character(subst_all$code))

vec_eff <- table(database$YEAR[!is.element(database$IMIS_SUBSTANCE_CODE,sub_list_all)])
reasons$N03[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[is.element(database$IMIS_SUBSTANCE_CODE,sub_list_all),] #restrict to subst in the list
length(unique(database$IMIS_SUBSTANCE_CODE)) #257
sort(unique(database$IMIS_SUBSTANCE_CODE))
length(unique(database$SUBSTANCE)) #281 (this number is higher than 257 because substance names are not standardized)
sort(unique(database$SUBSTANCE))

sum(reasons$N03,na.rm = TRUE)
length(database[,1]) #1387254(-118948)

[1] 978

[1] 265

[1] 257

[1] 257

[1] "0010" "0020" "0040" "0060" "0110" "0117" "0120" "0150" "0160" "0170"
 [11] "0190" "0227" "0230" "0260" "0270" "0290" "0310" "0320" "0360" "0372"
 [21] "0395" "0410" "0420" "0430" "0435" "0440" "0442" "0460" "0461" "0462"
 [31] "0471" "0472" "0477" "0490" "0491" "0520" "0524" "0527" "0529" "0530"
 [41] "0540" "0560" "0570" "0590" "0620" "0630" "0631" "0640" "0645" "0670"
 [51] "0680" "0681" "0685" "0686" "0689" "0690" "0692" "0700" "0710" "0720"
 [61] "0725" "0726" "0730" "0731" "0760" "0780" "0790" "0810" "0820" "0830"
 [71] "0860" "0867" "0868" "0874" "0907" "0915" "0921" "0924" "0928" "0930"
 [81] "0931" "1010" "1011" "1014" "1015" "1030" "1033" "1037" "1040" "1050"
 [91] "1060" "1073" "1080" "1170" "1190" "1210" "1280" "1285" "1290" "1300"
[101] "1310" "1325" "1330" "1340" "1361" "1371" "1377" "1380" "1385" "1420"
[111] "1430" "1440" "1460" "1470" "1480" "1490" "1515" "1520" "1534" "1536"
[121] "1538" "1539" "1540" "1560" "1591" "1610" "1618" "1620" "1631" "1643"
[131] "1647" "1650" "1653" "1660" "1675" "1680" "1690" "1720" "1730" "1732"
[141] "1740" "1750" "1770" "1774" "1776" "1790" "1791" "1797" "1810" "1840"
[151] "1842" "1860" "1890" "1903" "1911" "1941" "1942" "1943" "1944" "1947"
[161] "1948" "1949" "1950" "1953" "1980" "1989" "1990" "2000" "2010" "2020"
[171] "2037" "2038" "2040" "2070" "2085" "2110" "2140" "2150" "2170" "2180"
[181] "2210" "2215" "2216" "2217" "2230" "2240" "2260" "2270" "2280" "2290"
[191] "2310" "2390" "2430" "2431" "2440" "2460" "2470" "2480" "2485" "2490"
[201] "2495" "2505" "2536" "2540" "2571" "2572" "2580" "2582" "2590" "2610"
[211] "2611" "2650" "2651" "2720" "5010" "8120" "9000" "9010" "9015" "9017"
[221] "9020" "9130" "9135" "A100" "A102" "A110" "A624" "C110" "C141" "C730"
[231] "D129" "D668" "D740" "E105" "E108" "E115" "F118" "H130" "H146" "I200"
[241] "K107" "L129" "M111" "M139" "P125" "P136" "P218" "R251" "R290" "S103"
[251] "T177" "T185" "T405" "V107" "V109" "W103" "Z100"

[1] 281

[1] "1-Bromopropane"                                                        
  [2] "1-Methoxy-2-propanol"                                                  
  [3] "1-Methoxy-2-propyl acetate"                                            
  [4] "1,1,2-Trichloro-1,2,2-Trifluoroethane"                                 
  [5] "1,1,2-Trichloroethane"                                                 
  [6] "1,3,5-Triglycidyl Isocyanurate"                                        
  [7] "1,6-Hexamethylene Diisocyanate Homopolymer"                            
  [8] "135-Triglycidyl Isocyanurate"                                          
  [9] "16-Hexamethylene Diisocyanate Homopolymer"                             
 [10] "2-Butanone"                                                            
 [11] "2-Butoxyethanol"                                                       
 [12] "2-Ethoxyethanol"                                                       
 [13] "2-Ethoxyethyl Acetate"                                                 
 [14] "2-Hexanone"                                                            
 [15] "2-Methoxyethanol"                                                      
 [16] "2-Methoxyethyl acetate"                                                
 [17] "2-Nitropropane"                                                        
 [18] "2-Pentanone"                                                           
 [19] "2-Propoxyethanol"                                                      
 [20] "4,4'-Methylene-bis(2-Chloroaniline)"                                   
 [21] "4,4'-Methylenedianiline"                                               
 [22] "44'-Methylenedianiline"                                                
 [23] "Acetaldehyde"                                                          
 [24] "Acetic Acid"                                                           
 [25] "Acetone"                                                               
 [26] "Acetonitrile"                                                          
 [27] "Acetyl Methyl Carbinol"                                                
 [28] "Acrolein"                                                              
 [29] "Acrylic acid"                                                          
 [30] "Acrylonitrile"                                                         
 [31] "Allyl Propyl Disulfide"                                                
 [32] "alpha-Alumina (Total Dust)"                                            
 [33] "Aluminum (as Al) Metal (Respirable Fraction)"                          
 [34] "Aluminum (as Al) Metal (Total Dust)"                                   
 [35] "Aluminum (as Al) Welding Fumes"                                        
 [36] "Aluminum (as Al), Metal (Respirable Fraction)"                         
 [37] "Aluminum (as Al), Metal (Total Dust)"                                  
 [38] "Aluminum (as Al), Welding Fumes"                                       
 [39] "Ammonia"                                                               
 [40] "Anthracene"                                                            
 [41] "Antimony and Compounds (as Sb)"                                        
 [42] "Arsenic Inorganic"                                                     
 [43] "Arsenic, Inorganic"                                                    
 [44] "Arsine"                                                                
 [45] "Asbestos (all forms)"                                                  
 [46] "Asphalt Fumes (Petroleum)"                                             
 [47] "Barium, Soluble Compounds (as Ba)"                                     
 [48] "Benzene"                                                               
 [49] "Benzo [a] Pyrene"                                                      
 [50] "Beryllium and Beryllium Compounds (as Be)"                             
 [51] "beta-Chloroprene"                            

[1] 118948

[1] 1

In [8]:
# ##################### Preparation of conversion tables for QUALIFIER and UNIT_OF_MEASUREMENT ####

#### QUALIFIER
#### load table from 2011 ###/#
#qual.conv <- read.table("qual.conv.new.csv", header=T, sep=",", stringsAsFactor=F)
#table(qual.conv$clean)
# B       BLK eliminate        ND         S sample OK         W 
#19        51       161        73         6       332        10 
#write_xlsx(x = qual.conv, path = "qual.conv.xlsx", col_names = TRUE)

#### new 2020 table ###/#: creation of a new conversion table for QUALIFIER variable
# replace NAs in the qualifier variable by "raw was NA"
class(database$QUALIFIER)
length(database$QUALIFIER) #[1] 1387254
sum(is.na(database$QUALIFIER)) #[1] 751060
database$QUALIFIER[is.na(database$QUALIFIER)] <- "raw was NA"
qualif <- data.frame(table(database$QUALIFIER))
names(qualif) <-c('raw','n')
#nrow(qualif) 
sum(qualif$n) #[1] 1387254
#write_xlsx(x = qualif, path = "qualif_new.xlsx", col_names = TRUE)
qualif.conv.2020 <- read.csv(
    "Input\\Raw\\OSHA\\CEHD\\CEHD1984_2018\\cleaning scripts\\Conversion tables\\qualif_new_2020.csv",
    sep=";", header=T
)
qualif.conv.2020$clean <- as.character(qualif.conv.2020$clean)
qualif.conv.2020$raw <- as.character(qualif.conv.2020$raw)
qualif.conv.2020$possible_bulk <- as.character(qualif.conv.2020$possible_bulk)
table(qualif.conv.2020$clean)
#B       BLK  eliminate        ND         S sample OK         W 
#1         7        137        49         6        12         1 

[1] "character"

[1] 1387254

[1] 56403

[1] 1387254


        B       BLK eliminate        ND         S sample OK         W 
        1         7       137        49         6        12         1 

In [9]:
#### UNIT_OF_MEASUREMENT
#### load table from 2011 ###/#
#unit <- read.table("unit.new.csv", header=T, sep=",", stringsAsFactor=F)
#table(unit$clean)
#          % eliminate         F         M         P         X         Y 
#1         3        36         4         2         2         2         1
#write_xlsx(x = unit, path = "unit.new.xlsx", col_names = TRUE)

#### new 2020 table ###/#: creation of a new conversion table for UNIT_OF_MEASUREMENT variable
# replace NAs in the UNIT_OF_MEASUREMENT variable by "raw was NA"
class(database$UNIT_OF_MEASUREMENT)
length(database$UNIT_OF_MEASUREMENT) #[1] 1387254
sum(is.na(database$UNIT_OF_MEASUREMENT)) #[1] 29836
database$UNIT_OF_MEASUREMENT[is.na(database$UNIT_OF_MEASUREMENT)] <- "raw was NA"
unit_2020 <- data.frame(table(database$UNIT_OF_MEASUREMENT))
names(unit_2020) <-c('raw','n')
nrow(unit_2020) 
sum(unit_2020$n) #[1] 1387254
#write_xlsx(x = unit_2020, path = "unit_conv.xlsx", col_names = TRUE)
unit.conv.2020 <- read.csv(
    "Input\\Raw\\OSHA\\CEHD\\CEHD1984_2018\\cleaning scripts\\Conversion tables\\unit_conv_2020.csv",
    sep=";", header=T
)
unit.conv.2020$clean <- as.character(unit.conv.2020$clean)
unit.conv.2020$raw <- as.character(unit.conv.2020$raw)
table(unit.conv.2020$clean)
#          % eliminate         F         M         P         X         Y 
#2         1        22         4         2         2         2         1


##Add a column that indicates that the sample is censored ONLY based on the Qualifier variable with
#approximative '<' signs
database$CENSORED <- rep('N',length(database$IMIS_SUBSTANCE_CODE))
database$CENSORED[database$QUALIFIER=="-<"] <- "Y"
database$CENSORED[database$QUALIFIER=="  <"] <- "Y"
database$CENSORED[database$QUALIFIER==" =<"] <- "Y"
database$CENSORED[database$QUALIFIER=="@<"] <- "Y"
database$CENSORED[database$QUALIFIER=="@<="] <- "Y"
database$CENSORED[database$QUALIFIER=="@=<"] <- "Y"
database$CENSORED[database$QUALIFIER=="<"] <- "Y"
database$CENSORED[database$QUALIFIER=="< ="] <- "Y"
database$CENSORED[database$QUALIFIER=="<@"] <- "Y"
database$CENSORED[database$QUALIFIER=="<="] <- "Y"
database$CENSORED[database$QUALIFIER=="<= 0"] <- "Y"
database$CENSORED[database$QUALIFIER=="= <"] <- "Y"
database$CENSORED[database$QUALIFIER=="=<"] <- "Y"
database$CENSORED[database$QUALIFIER=="=<@"] <- "Y"


# "raw was NA" transformed into EMPTY in QUALIFIER
length(database$QUALIFIER[is.na(database$QUALIFIER)]) #0
length(database$QUALIFIER[database$QUALIFIER=="raw was NA"]) #751060
database$QUALIFIER[database$QUALIFIER=="raw was NA"] <- ''
length(database$QUALIFIER[database$QUALIFIER=="raw was NA"]) #0

# NAs transformed into 0 in SAMPLE_RESULT
database$SAMPLE_RESULT_2 <- database$SAMPLE_RESULT
length(database$SAMPLE_RESULT_2[is.na(database$SAMPLE_RESULT_2)]) #39131
database$SAMPLE_RESULT_2[is.na(database$SAMPLE_RESULT_2)] <- 0
length(database$SAMPLE_RESULT_2[is.na(database$SAMPLE_RESULT_2)]) #0

[1] "character"

[1] 1387254

[1] 0

[1] 35

[1] 1387254

...

[1] 0

[1] 56403

[1] 0

[1] 39131

[1] 0

In [10]:
##################### Standardisation of qualifier variable ##################### 
#####Removing samples: Qualifier suggests ND but result is >0
# ##################### N08 / N29 qualifier suggests ND but sample result is not null ####
# N08
temp <- database[(database$SAMPLE_RESULT_2>0 & database$CENSORED!="Y" &
                            is.element(database$QUALIFIER, c(qualif.conv.2020$raw[qualif.conv.2020$clean=="ND"]))),]

vec_eff <- table(database$YEAR[(database$SAMPLE_RESULT_2>0 & database$CENSORED!="Y" &
                                is.element(database$QUALIFIER, c(qualif.conv.2020$raw[qualif.conv.2020$clean=="ND"])))])
reasons$N08[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!(database$SAMPLE_RESULT_2>0 & database$CENSORED!="Y" &
                      is.element(database$QUALIFIER, c(qualif.conv.2020$raw[qualif.conv.2020$clean=="ND"]))), ] #exclusion
sum(reasons$N08,na.rm = TRUE)
length(database[,1]) #1386387(-867)

# N29
temp <- database[(database$SAMPLE_RESULT_2>0 & (database$CENSORED=="Y" | 
               is.element(database$QUALIFIER, c(qualif.conv.2020$raw[qualif.conv.2020$clean=="ND"])))),]

vec_eff <- table(database$YEAR[(database$SAMPLE_RESULT_2>0 & (database$CENSORED=="Y" | 
               is.element(database$QUALIFIER, c(qualif.conv.2020$raw[qualif.conv.2020$clean=="ND"]))))])
reasons$N29[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!(database$SAMPLE_RESULT_2>0 & (database$CENSORED=="Y" | 
                     is.element(database$QUALIFIER, c(qualif.conv.2020$raw[qualif.conv.2020$clean=="ND"])))),] #exclusion
sum(reasons$N29,na.rm = TRUE)
length(database[,1]) #1377502(-8885)

[1] 867

[1] 1

[1] 8885

[1] 1

In [11]:
#####Removing samples: Qualifier conflicting with unit of measurement / sample type, or judged 'to be eliminated'
#cleaning unit of measurement

database$UNIT_OF_MEASUREMENT_2 <- database$UNIT_OF_MEASUREMENT
unik.clean <- unique(unit.conv.2020$clean)

for (j in 1:length(unik.clean))
{
one.clean <- as.character(unik.clean)[j]
raw.values <- as.character(unit.conv.2020$raw[unit.conv.2020$clean==one.clean])
database$UNIT_OF_MEASUREMENT_2[is.element(database$UNIT_OF_MEASUREMENT, raw.values)] <- one.clean 
}

as.data.frame(table(database$UNIT_OF_MEASUREMENT))
table(database$UNIT_OF_MEASUREMENT_2)

Var1,Freq
<fct>,<int>
',1
%,36789
",",1
.,1
@,1
[,1
\,2
],1
0,10



        % eliminate         F         M      None         P         X         Y 
    36789      2015     18951   1020239     29840    267442      2096       130 

In [12]:
# ##################### N04: qualifier=BLK and not possible bulk # elimination ####
temp <- database[is.element(database$QUALIFIER, c(qualif.conv.2020$raw[qualif.conv.2020$clean=="BLK" & 
                                                                       qualif.conv.2020$possible_bulk=="N"])),]

vec_eff <- table(database$YEAR[is.element(database$QUALIFIER, c(qualif.conv.2020$raw[qualif.conv.2020$clean=="BLK" & 
                                                                                     qualif.conv.2020$possible_bulk=="N"]))])
reasons$N04[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!is.element(database$QUALIFIER, c(qualif.conv.2020$raw[qualif.conv.2020$clean=="BLK" & 
                                                                            qualif.conv.2020$possible_bulk=="N"])),] #exclusion
sum(reasons$N04,na.rm = TRUE)
length(database[,1]) #1377481(-21)

[1] 21

[1] 1

In [13]:
# ##################### N05: qualifier deemed not interpretable and record is to be eliminated ####
temp <- database[is.element(database$QUALIFIER, c(qualif.conv.2020$raw[qualif.conv.2020$clean=="eliminate"])),]

vec_eff <- table(database$YEAR[is.element(database$QUALIFIER, c(qualif.conv.2020$raw[qualif.conv.2020$clean=="eliminate"]))])
reasons$N05[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!is.element(database$QUALIFIER, c(qualif.conv.2020$raw[qualif.conv.2020$clean=="eliminate"])),] #exclusion
sum(reasons$N05,na.rm = TRUE)
length(database[,1]) #1377022(-459)

[1] 458

[1] 1

In [14]:
# ##################### N06 / N07: qualifier conflicting with sample type ####
temp <- database[is.element(database$QUALIFIER, c(qualif.conv.2020$raw[qualif.conv.2020$clean=="B" | qualif.conv.2020$clean=="W"])),]

vec_eff <- table(database$YEAR[is.element(database$QUALIFIER, c(qualif.conv.2020$raw[qualif.conv.2020$clean=="B" | qualif.conv.2020$clean=="W"]))])
reasons$N0607[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!is.element(database$QUALIFIER, c(qualif.conv.2020$raw[qualif.conv.2020$clean=="B" | qualif.conv.2020$clean=="W"])),] #exclusion
sum(reasons$N0607,na.rm = TRUE)
length(database[,1]) #1376771(-251)

[1] 251

[1] 1

In [15]:
# ##################### N09: qualifier=BLK and a possible bulk as judged by researcher and variable blank_used says NO and sample type is not bulk, final result=eliminate ####
temp <- database[is.element(database$QUALIFIER, c(qualif.conv.2020$raw[qualif.conv.2020$clean=="BLK" & qualif.conv.2020$possible_bulk=="Y"])) & is.element(database$BLANK_USED,"N"),]

vec_eff <- table(database$YEAR[is.element(database$QUALIFIER, c(qualif.conv.2020$raw[qualif.conv.2020$clean=="BLK" & qualif.conv.2020$possible_bulk=="Y"])) & is.element(database$BLANK_USED,"N")])
reasons$N09[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!is.element(database$QUALIFIER, c(qualif.conv.2020$raw[qualif.conv.2020$clean=="BLK" & qualif.conv.2020$possible_bulk=="Y"])) & is.element(database$BLANK_USED,"N"),] #exclusion
sum(reasons$N09,na.rm = TRUE)
length(database[,1]) #1376453(-318)

[1] 318

[1] 1

In [16]:
# ##################### N10: combustion related : eliminate ####
temp <- database[is.element(database$QUALIFIER, c("COMB",'COMD','com','comb')),]

vec_eff <- table(database$YEAR[is.element(database$QUALIFIER, c("COMB",'COMD','com','comb'))])
reasons$N10[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!is.element(database$QUALIFIER, c("COMB",'COMD','com','comb')),] #exclusion
sum(reasons$N10,na.rm = TRUE)
length(database[,1]) #1376258(-195)

[1] 195

[1] 1

In [17]:
# ##################### N11: qualifier suggest fibers (F) but F not relevant for substance ####
temp <- database[database$QUALIFIER=="F" & database$IMIS_SUBSTANCE_CODE!=9020,]

vec_eff <- table(database$YEAR[database$QUALIFIER=="F" & database$IMIS_SUBSTANCE_CODE!=9020])
reasons$N11[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!(database$QUALIFIER=="F" & database$IMIS_SUBSTANCE_CODE!=9020),] #exclusion
sum(reasons$N11,na.rm = TRUE)
length(database[,1]) #1376256(-2)

[1] 2

[1] 1

In [18]:
# ##################### N13: 'Y' qualifier value judged possible 'Ytrium', deemed OK for the generic particles category only : 9135 ####
temp <- database[database$QUALIFIER=="Y" & database$IMIS_SUBSTANCE_CODE!=9135,]

vec_eff <- table(database$YEAR[database$QUALIFIER=="Y" & database$IMIS_SUBSTANCE_CODE!=9135])
reasons$N13[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!(database$QUALIFIER=="Y" & database$IMIS_SUBSTANCE_CODE!=9135),] #exclusion
sum(reasons$N13,na.rm = TRUE)
length(database[,1]) #1376206(-50)

[1] 50

[1] 1

In [19]:
# ##################### N16: measure is approximate (to be eliminated) ####
temp <- database[is.element(database$QUALIFIER,c("@"," @","@<","@=<","@<=","<@","=<@","EST")),]

vec_eff <- table(database$YEAR[is.element(database$QUALIFIER,c("@"," @","@<","@=<","@<=","<@","=<@","EST"))])
reasons$N16[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!is.element(database$QUALIFIER,c("@"," @","@<","@=<","@<=","<@","=<@","EST")),] #exclusion
sum(reasons$N16,na.rm = TRUE)
length(database[,1]) #1375371(-835)

[1] 835

[1] 1

In [20]:
# ##################### N19: elimination of records that have "%" in qualifier, but not "%" in the unit (to be removed), same for "M"  ####
temp <- database[(database$UNIT_OF_MEASUREMENT_2!="%" & database$QUALIFIER=="%") | (database$UNIT_OF_MEASUREMENT_2!="M" & database$QUALIFIER=="M"),]

vec_eff <- table(database$YEAR[(database$UNIT_OF_MEASUREMENT_2!="%" & database$QUALIFIER=="%") | (database$UNIT_OF_MEASUREMENT_2!="M" & database$QUALIFIER=="M")])
reasons$N19[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!((database$UNIT_OF_MEASUREMENT_2!="%" & database$QUALIFIER=="%") | (database$UNIT_OF_MEASUREMENT_2!="M" & database$QUALIFIER=="M")),] #exclusion
sum(reasons$N19,na.rm = TRUE)
length(database[,1]) #1375365(-6)

[1] 6

[1] 1

In [21]:
#####Removing samples: Unit of measurement is judged erroneous / missing  but the sample result is not null / conflicting with substance

# ##################### N17 / N18: elimination of records that should not have "F" as unit  ####
temp <- database[database$UNIT_OF_MEASUREMENT_2=="F",]
class(temp$IMIS_SUBSTANCE_CODE)
temp$IMIS_SUBSTANCE_CODE <- as.character(temp$IMIS_SUBSTANCE_CODE)
table(temp$IMIS_SUBSTANCE_CODE)
#0527  1073  1300  2270  2470  9020  9135  R251 
#   1     5   576     1     1 18105     1   189  

temp <- database[is.element(database$IMIS_SUBSTANCE_CODE,c('0527','1073','2270','2470','9135')) & database$UNIT_OF_MEASUREMENT_2=="F",]

vec_eff <- table(database$YEAR[is.element(database$IMIS_SUBSTANCE_CODE,c('1073','2270','2470','9135')) & database$UNIT_OF_MEASUREMENT_2=="F"])
reasons$N1718[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!(is.element(database$IMIS_SUBSTANCE_CODE,c('1073','2270','2470','9135')) & database$UNIT_OF_MEASUREMENT_2=="F"),] #exclusion
sum(reasons$N1718,na.rm = TRUE)
length(database[,1]) #1375357(-8)

[1] "character"


 0527  1073  1300  2270  2470  9020  9135  R251 
    1     5   576     1     1 18105     1   189 

[1] 8

[1] 1

In [22]:
# ##################### N23: elimination of records for which the sample result is not null and that the unit is empty  ####
temp <- database[database$UNIT_OF_MEASUREMENT_2=="" & database$SAMPLE_RESULT_2>0,]

vec_eff <- table(database$YEAR[database$UNIT_OF_MEASUREMENT_2=="" & database$SAMPLE_RESULT_2>0])
reasons$N23[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!(database$UNIT_OF_MEASUREMENT_2=="" & database$SAMPLE_RESULT_2>0),] #exclusion
sum(reasons$N23,na.rm = TRUE)
length(database[,1]) #1374125(-1232)

[1] 0

[1] 1

In [23]:
# ##################### N24: elimination of cases where unit is "%" in unit, but sample_result is >100  ####
temp <- database[database$UNIT_OF_MEASUREMENT_2=="%" & database$SAMPLE_RESULT_2>100,]

vec_eff <- table(database$YEAR[database$UNIT_OF_MEASUREMENT_2=="%" & database$SAMPLE_RESULT_2>100])
reasons$N24[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!(database$UNIT_OF_MEASUREMENT_2=="%" & database$SAMPLE_RESULT_2>100),] #exclusion
sum(reasons$N24,na.rm = TRUE)
length(database[,1]) #1374117(-8)


## Modification of variables qualifier and air volume sampled ####
##creating a qualifier.2 : detected / not detected
database$QUALIFIER_2 <- rep('detected',length(database[,1]))
database$QUALIFIER_2[database$SAMPLE_RESULT_2==0] <- 'ND'	

##standardizing air volume sampled
#sampvol.conv <-read.table("sampvol.conv.new.csv", header=T, sep=",", stringsAsFactor=F)
#sampvol.conv.1 <-read.table("sampvol.conv.1.new.csv", header=T, sep=",", stringsAsFactor=F)
#sampvol.conv.rest <-read.table("sampvol.conv.rest.new.csv", header=T, sep=",", stringsAsFactor=F)
#sampvol.conv.rest.1 <-read.table("sampvol.conv.rest.1.new.csv", header=T, sep=",", stringsAsFactor=F)
#sampvol.conv.tot <-rbind(sampvol.conv,sampvol.conv.rest,sampvol.conv.1,sampvol.conv.rest.1)
#write_xlsx(x = sampvol.conv.tot, path = "sampvol.conv.tot.xlsx", col_names = TRUE)

#**** this step is simplified for the 2020 cleanup: AIR_VOLUME_SAMPLED==NA, 0 or empty are deleted at N22 / N27 **** 
class(database$AIR_VOLUME_SAMPLED)
length(database$AIR_VOLUME_SAMPLED[is.na(database$AIR_VOLUME_SAMPLED)]) #[1] 3897
temp <- database[which(is.na(as.numeric(as.character(database$AIR_VOLUME_SAMPLED)))),]
summary(database$AIR_VOLUME_SAMPLED,na.rm=TRUE)
#   Min.  1st Qu.   Median     Mean  3rd Qu.     Max.     NA's 
#   0.0     30.8    314.4    368.9    606.0 924560.0     3897 
cehd_top <- database[1:5000,]


#further standardization of units
#based on the units for the personal cleaned, we keep only the %, F, M, P data
#in addition sample weight must not be null for detected results with perc

#list of 29 substances with most records
subst <- data.frame(table(database$IMIS_SUBSTANCE_CODE))
names(subst) <-c('code','n')
subst <- subst[order(-subst$n),]
nrow(subst) #[1] 257
list_29 <- as.character(sort(subst$code[c(1:29)]))
list_29

#list used in 2011 cleanup
substances.list.29 <-c('0040','0230','0260','0360','0430','0491','0685','0720','0731','1073','1290','1520','1560',
                     '1591','1620','1730','1790','1840','2270','2280','2460','2571','2590','2610','9020','9130','9135','C141','S103')

setdiff(list_29,substances.list.29) #[1] "9010"
setdiff(substances.list.29,list_29) #[1] "S103"
#9010 is replaced by S103 in the list
sum(subst$n[is.element(subst$code,substances.list.29)]) #[1] 1099503

#list used in 2011 cleanup will also be used in the current 2020 cleanup

[1] 8

[1] 1

[1] "numeric"

[1] 3925

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max.     NA's 
     0.0     30.8    314.2    368.8    606.0 924560.0     3925 

[1] 257

[1] "0040" "0230" "0260" "0360" "0430" "0491" "0685" "0720" "0731" "1073"
[11] "1290" "1520" "1560" "1591" "1620" "1730" "1790" "1840" "2270" "2280"
[21] "2460" "2571" "2590" "2610" "9010" "9020" "9130" "9135" "C141"

[1] "9010"

[1] "S103"

[1] 1100427

In [24]:
# ##################### N31: record from the '29' (with 9010 excluded) and unit is not part of '','F','P','M'  ####
#table(database$UNIT_OF_MEASUREMENT_2)
temp <- database[(is.element(database$IMIS_SUBSTANCE_CODE,substances.list.29)) &
                (!is.element(database$UNIT_OF_MEASUREMENT_2,c('','F','P','M'))),]

vec_eff <- table(database$YEAR[(is.element(database$IMIS_SUBSTANCE_CODE,substances.list.29)) &
                               (!is.element(database$UNIT_OF_MEASUREMENT_2,c('','F','P','M')))])
reasons$N31[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!((is.element(database$IMIS_SUBSTANCE_CODE,substances.list.29)) &
                     (!is.element(database$UNIT_OF_MEASUREMENT_2,c('','F','P','M')))),] #exclusion
sum(reasons$N31,na.rm = TRUE)
length(database[,1]) #1371807(-2310)
#temp <- database[database$UNIT_OF_MEASUREMENT_2=="%",]

[1] 14125

[1] 1

In [25]:
# ##################### N32: record from substance '9010' : if not '','%' or 'M' eliminate  ####
table(database$UNIT_OF_MEASUREMENT_2)

temp <- database[(is.element(database$IMIS_SUBSTANCE_CODE,'9010')) &
                 (!is.element(database$UNIT_OF_MEASUREMENT_2,c('','%','M'))),]

vec_eff <- table(database$YEAR[(is.element(database$IMIS_SUBSTANCE_CODE,'9010')) &
                               (!is.element(database$UNIT_OF_MEASUREMENT_2,c('','%','M')))])
reasons$N32[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!((is.element(database$IMIS_SUBSTANCE_CODE,'9010')) &
                       (!is.element(database$UNIT_OF_MEASUREMENT_2,c('','%','M')))),] #exclusion
sum(reasons$N32,na.rm = TRUE)
length(database[,1]) #1371631(-176)
#temp <- database[database$UNIT_OF_MEASUREMENT_2=="%",]


        % eliminate         F         M      None         P         X         Y 
    35329      1639     18871   1019691     17554    267242       828        71 

[1] 4336

[1] 1

In [26]:
# ##################### N33: record not from 29 substances or  '9010' : if not '','P','F','%' or 'M' eliminate  ####
#table(database$UNIT_OF_MEASUREMENT_2)
restrict <- !database$IMIS_SUBSTANCE_CODE=='9010' & !is.element(database$IMIS_SUBSTANCE_CODE,substances.list.29)
temp <- database[restrict & (!is.element(database$UNIT_OF_MEASUREMENT_2,c('','%','M','P','F'))),]

vec_eff <- table(database$YEAR[restrict & (!is.element(database$UNIT_OF_MEASUREMENT_2,c('','%','M','P','F')))])
reasons$N33[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!(restrict & (!is.element(database$UNIT_OF_MEASUREMENT_2,c('','%','M','P','F')))),] #exclusion
sum(reasons$N33,na.rm = TRUE)
length(database[,1]) #1369269(-2362)
#temp <- database[database$UNIT_OF_MEASUREMENT_2=="%",]

[1] 15756

[1] 1

In [27]:
# ##################### N34: elimination when unit is perc (%) and the result is non null but sample weight is null ####
#missing sampling weight replaced by 0
database$SAMPLE_WEIGHT_2 <- database$SAMPLE_WEIGHT
length(database$SAMPLE_WEIGHT_2[is.na(database$SAMPLE_WEIGHT)])
database$SAMPLE_WEIGHT_2[is.na(database$SAMPLE_WEIGHT)] <- 0
length(database$SAMPLE_WEIGHT_2[is.na(database$SAMPLE_WEIGHT_2)])

temp <- database[database$SAMPLE_WEIGHT_2==0 & database$UNIT_OF_MEASUREMENT_2=="%" & 
               database$SAMPLE_RESULT_2>0,]

vec_eff <- table(database$YEAR[database$SAMPLE_WEIGHT_2==0 & database$UNIT_OF_MEASUREMENT_2=="%" & 
                               database$SAMPLE_RESULT_2>0])
reasons$N34[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!(database$SAMPLE_WEIGHT_2==0 & database$UNIT_OF_MEASUREMENT_2=="%" & 
                       database$SAMPLE_RESULT_2>0),] #exclusion
sum(reasons$N34,na.rm = TRUE)
length(database[,1]) #1366423(-2846)

#transformation of percs into mg/m3										
length(database$SAMPLE_WEIGHT_2[is.na(database$SAMPLE_WEIGHT_2)]) #[1] 0
length(database$AIR_VOLUME_SAMPLED[is.na(database$AIR_VOLUME_SAMPLED)]) #[1] 3309
table(database$UNIT_OF_MEASUREMENT_2)
#            %       F       M       P 
#28136   32483   18871 1019691  267242

restrict.1 <- database$SAMPLE_WEIGHT_2!=0 &
database$UNIT_OF_MEASUREMENT_2=='%' &
database$SAMPLE_RESULT_2>0 &
!is.na(database$SAMPLE_WEIGHT_2) &
!is.na(database$AIR_VOLUME_SAMPLED) & database$AIR_VOLUME_SAMPLED>0

database$SAMPLE_RESULT_3 <- database$SAMPLE_RESULT_2

database$SAMPLE_RESULT_3[restrict.1] <- database$SAMPLE_RESULT_2[restrict.1]*
database$SAMPLE_WEIGHT_2[restrict.1]*10/
database$AIR_VOLUME_SAMPLED[restrict.1]										

database$UNIT_OF_MEASUREMENT_2[restrict.1] <- 'M.from.Perc'
temp <- database[database$UNIT_OF_MEASUREMENT_2=="P",]

table(database$UNIT_OF_MEASUREMENT_2)
#                %           F           M M.from.Perc           P 
#28136       16779       18871     1019691       15704      26724 

[1] 852780

[1] 0

[1] 2846

[1] 1

[1] 0

[1] 3091


      %       F       M       P 
  32483   18871 1019691  267242 


          %           F           M M.from.Perc           P 
      16779       18871     1019691       15704      267242 

In [28]:
# ##################### N20: elimination of the records that have a missing value for the office ID  ####
temp <- database[is.na(database$OFFICE_ID),]

vec_eff <- table(database$YEAR[is.na(database$OFFICE_ID)])
reasons$N20[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!is.na(database$OFFICE_ID),] #exclusion
sum(reasons$N20,na.rm = TRUE)
length(database[,1]) #1363807(-2616)

[1] 2239

[1] 1

In [29]:
# ##################### N21: records that have a missing time_sampled variable are deleted  ####
temp <- database[is.na(database$TIME_SAMPLED),]

vec_eff <- table(database$YEAR[is.na(database$TIME_SAMPLED)])
reasons$N21[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!is.na(database$TIME_SAMPLED),] #exclusion
sum(reasons$N21,na.rm = TRUE)
length(database[,1]) #1362164(-1643)

[1] 1490

[1] 1

In [30]:
# ##################### N26: elimination of the records that have a null time_sampled variable   ####
temp <- database[database$TIME_SAMPLED==0,]

vec_eff <- table(database$YEAR[database$TIME_SAMPLED==0])
reasons$N26[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[database$TIME_SAMPLED!=0,] #exclusion
sum(reasons$N26,na.rm = TRUE)
length(database[,1]) #1357348(-4816)

[1] 4516

[1] 1

In [31]:
# ##################### N28: elimination of sample results less than 0   ####
temp <- database[database$SAMPLE_RESULT_3<0,]

vec_eff <- table(database$YEAR[database$SAMPLE_RESULT_3<0])
reasons$N28[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[database$SAMPLE_RESULT_3>=0,] #exclusion
sum(reasons$N28,na.rm = TRUE)
length(database[,1]) #1357347(-1)

[1] 1

[1] 1

In [32]:
# ##################### N25: records to be deleted because of a missing or null sampling number   ####
temp <- database[is.na(database$SAMPLING_NUMBER) | database$SAMPLING_NUMBER==0,]

vec_eff <- table(database$YEAR[is.na(database$SAMPLING_NUMBER) | database$SAMPLING_NUMBER==0])
reasons$N25[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!(is.na(database$SAMPLING_NUMBER) | database$SAMPLING_NUMBER==0),] #exclusion
sum(reasons$N25,na.rm = TRUE)
length(database[,1]) #1355916(-1431)

[1] 1378

[1] 1

In [33]:
# ##################### N22: records that have a NA or '' volume sampled variable are deleted   ####
summary(database$AIR_VOLUME_SAMPLED)
#  Min.  1st Qu.   Median     Mean  3rd Qu.     Max.     NA's 
#   0.0     32.1    316.6    370.3    608.9 924560.0     1705 
temp <- database[is.na(database$AIR_VOLUME_SAMPLED),] #1705
temp <- database[is.na(database$AIR_VOLUME_SAMPLED) | database$AIR_VOLUME_SAMPLED=='',] #1705

vec_eff <- table(database$YEAR[is.na(database$AIR_VOLUME_SAMPLED) | database$AIR_VOLUME_SAMPLED==''])
reasons$N22[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!(is.na(database$AIR_VOLUME_SAMPLED) | database$AIR_VOLUME_SAMPLED==''),] #exclusion
sum(reasons$N22,na.rm = TRUE)
length(database[,1]) #1354211(-1705)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max.     NA's 
     0.0     36.0    320.0    372.8    613.0 924560.0     1642 

[1] 1642

[1] 1

In [34]:
# ##################### N27: elimination of records that have an air volume sampled of zero   ####
summary(database$AIR_VOLUME_SAMPLED)
#  Min.  1st Qu.   Median     Mean  3rd Qu.     Max.     
#   0.0     32.1    316.6    370.3    608.9 924560.0     
temp <- database[database$AIR_VOLUME_SAMPLED==0,] 

vec_eff <- table(database$YEAR[database$AIR_VOLUME_SAMPLED==0])
reasons$N27[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[database$AIR_VOLUME_SAMPLED>0,] #exclusion
sum(reasons$N27,na.rm = TRUE)
length(database[,1]) #1341727(-12484)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
     0.0     36.0    320.0    372.8    613.0 924560.0 

[1] 11998

[1] 1

In [35]:
# ##################### N35: instrument type is equal to ''   ####
#temp <- database[database$INSTRUMENT_TYPE=='' & !is.na(database$INSTRUMENT_TYPE),] 
#temp <- database[is.na(database$INSTRUMENT_TYPE),] 
#table(temp$YEAR)

restrictN35 <- database$INSTRUMENT_TYPE=='' & !is.na(database$INSTRUMENT_TYPE)
#table(restrictN35)

temp <- database[restrictN35,]

vec_eff <- table(database$YEAR[restrictN35])
reasons$N35[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!restrictN35,] #exclusion
sum(reasons$N35,na.rm = TRUE)
length(database[,1]) #1341727(-0)

[1] 0

[1] 1

In [36]:
# ##################### N37: instrument type is missing   ####
#****NAs are not removed at this step: they are treated at N36**** 
temp <- database[is.na(database$INSTRUMENT_TYPE),] 
table(temp$YEAR)
restrictN37 <- is.na(database$INSTRUMENT_TYPE)
table(restrictN37)
#  FALSE    TRUE 
#1171397  170330 


 1984  1985  1986  1987  1988  1989  1990  1991  1992  1993  1994  1995  1996 
    0     0     1     1     0     0     0     0     2     0     0     0     0 
 1997  1998  1999  2000  2001  2002  2003  2004  2005  2006  2007  2008  2009 
    0     0     0     0     0     0     0     0     0     0     0     0     0 
 2010  2011  2013  2014  2015  2016  2017  2018 
    0     0 30796 32833 30048 29147 26725 19599 

restrictN37
  FALSE    TRUE 
1145871  169152 

In [37]:
# ##################### N36: cleaning of instrument type ####
database$INSTRUMENT_TYPE[is.na(database$INSTRUMENT_TYPE)] <- ''
length(database$INSTRUMENT_TYPE[database$INSTRUMENT_TYPE=='']) #[1] 170330
temp <- database[database$INSTRUMENT_TYPE=='',]

### cleaning strategy ###
#original 81 substances: clean for 1984-2009 using IT tables
#original 81 substances: copy raw IT for 2010-2011
#new substances: copy raw IT for 1984-2011
#all substances: indicate "not recorded" for 2013-2018

#1) all substances: create INSTRUMENT_TYPE_2 variable + indicate "not recorded" for all years (1984-2018)
database$INSTRUMENT_TYPE_2 <- 'not recorded'

#2) all substances: copy raw IT for 1984-2011
database$INSTRUMENT_TYPE_2[as.numeric(as.character(database$YEAR))<2012] <- database$INSTRUMENT_TYPE[as.numeric(as.character(database$YEAR))<2012]
temp <- database[database$INSTRUMENT_TYPE_2=='not recorded',]
table(temp$YEAR)

#3) original 81 substances: clean for 1984-2009 using IT tables
#store tables in a list object
conv.tables.path = "Input\\Raw\\OSHA\\CEHD\\CEHD1984_2018\\cleaning scripts\\Conversion tables IT"
files <- list.files(conv.tables.path)
n <- length(files)
data.list <- vector(mode="list",n)

for (i in 1:n) {
subs_code <- substr(files[i],3,6)
csv.path = file.path(conv.tables.path, files[i])
data.list[[i]] <- read.csv(csv.path,sep=",", header=TRUE)
names(data.list)[i] <- subs_code
print(i)
}

data.list[i]

length(database$IMIS_SUBSTANCE_CODE[database$IMIS_SUBSTANCE_CODE=="0040"])
#[1] 10726
length(database$IMIS_SUBSTANCE_CODE[database$IMIS_SUBSTANCE_CODE=="0040" & as.numeric(as.character(database$YEAR))<2010])
#[1] 9691

for (i in 1:n) {

restrict <- database$IMIS_SUBSTANCE_CODE==names(data.list)[i] & as.numeric(as.character(database$YEAR))<2010
table(restrict)

unik.clean <- unique(data.list[[i]]$clean)

for (j in 1:length(unik.clean))
{
  one.clean <- as.character(unik.clean)[j]
  raw.values <- as.character(data.list[[i]]$raw[data.list[[i]]$clean==one.clean])
  
  database$INSTRUMENT_TYPE_2[restrict & is.element(database$INSTRUMENT_TYPE,raw.values)] <- one.clean
 }

print(i)
}

#verifications
#View(database[,c(9,11,22,26,32,33)])
#View(database[database$INSTRUMENT_TYPE=='',c(9,11,22,26,32,33)])
#View(database[database$INSTRUMENT_TYPE_2=='',c(9,11,22,26,32,33)])

temp <- database[database$INSTRUMENT_TYPE_2=='',]
table(temp$YEAR)

temp <- database[database$IMIS_SUBSTANCE_CODE=="9135",c(9,11,26,32,33)]
temp <- database[database$IMIS_SUBSTANCE_CODE=="9135" & database$INSTRUMENT_TYPE=='',c(9,11,26,32,33)]
table(temp$YEAR)

#4) replace INSTRUMENT_TYPE_2=='' by INSTRUMENT_TYPE_2=='eliminate' (these records correspond to 
#   new substances <2012 which didn't have an IT recorded)
length(database$INSTRUMENT_TYPE_2[database$INSTRUMENT_TYPE_2=='']) #[1] 129
database$INSTRUMENT_TYPE_2[database$INSTRUMENT_TYPE_2==''] <- 'eliminate'


#exclusion
restrictN36 <- database$INSTRUMENT_TYPE_2=='eliminate'
table(restrictN36)
#  FALSE    TRUE 
#1330338   11389

temp <- database[restrictN36,]

vec_eff <- table(database$YEAR[restrictN36])
reasons$N36[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database <- database[!restrictN36,] #exclusion
sum(reasons$N36,na.rm = TRUE)
length(database[,1]) #1330338(-11389)

[1] 169152


 1984  1985  1986  1987  1988  1989  1990  1991  1992  1993  1994  1995  1996 
    0     0     0     0     0     0     0     0     0     0     0     0     0 
 1997  1998  1999  2000  2001  2002  2003  2004  2005  2006  2007  2008  2009 
    0     0     0     0     0     0     0     0     0     0     0     0     0 
 2010  2011  2013  2014  2015  2016  2017  2018 
    0     0 30796 32833 30048 29147 26725 19599 

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81


,raw,number_of_records,clean
,<chr>,<int>,<chr>
1,GC,267,GC-FID
2,GC-FID,2178,GC-FID
3,GC-FID OSHA 7,3,7
4,GC-IFD,3,GC-FID
5,GC/FD,6,GC-FID
6,GC/FID,2985,GC-FID
7,GHC/FID,10,GC-FID
8,GIVEN TO INORGAN,1,eliminate
9,TBD,27,eliminate


[1] 10456

...


1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 
   0    0    0    0    0    0    0    0    2    0    0    0    0    0    0    0 
2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2013 2014 2015 2016 
   0    0    0    0    0    0    0    0    0    0    0    0  844 1060  938 1103 
2017 2018 
 898  835 

[1] 0

restrictN36
  FALSE    TRUE 
1305423    9600 

[1] 9600

[1] 1

In [38]:
# ##################### Duplicates ####
names(database)
#creating the hash variable
database$HASH <- paste(database$INSPECTION_NUMBER,
                     database$IMIS_SUBSTANCE_CODE,
                     database$SAMPLING_NUMBER,
                     database$FIELD_NUMBER,sep='-')

#file listing the problematic hash codes					
bla <- data.frame(table(database$HASH),stringsAsFactors=F)
names(bla) <- c('name','n')
bla <- bla[bla$n>1,]
bla$name <- as.character(bla$name)
bla$code <- database$IMIS_SUBSTANCE_CODE[match(bla$name,database$HASH)]
bla$sub <- database$SUBSTANCE[match(bla$name,database$HASH)]
length(bla$name) #[1] 6587
sum(bla$n) #[1] 13197

#creating a concatenated variable to identify if there are true duplicates
database$CONCAT <- paste(database$LAB_NUMBER,
                       database$STATE,
                       database$ZIP_CODE,
                       database$YEAR,
                       database$TIME_SAMPLED,
                       database$SAMPLE_WEIGHT_2,
                       sep='-')


#identification of cases where "CONCAT" is the same 
bla$concatdiff <- rep(F,length(bla[,1]))

for (i in 1:length(bla[,1])) {

if (length(unique(database$CONCAT[database$HASH==bla$name[i]]))!=1) bla$concatdiff[i] <- T 
print(i)

}

table(bla$concatdiff)

bla$name
bla$name[1]
temp <- database[database$HASH=="100014786-9010-1179480-HLR1",]

table(bla$concatdiff)
#FALSE  TRUE 
# 3136  3451 
sum(bla$n) #[1] 13197
sum(bla$n[bla$concatdiff==FALSE]) #[1] 6272
sum(bla$n[bla$concatdiff==TRUE]) #[1] 6925

#### M: false duplicates ####
#list of hash codes to eliminate because "CONCAT" variable varies
Hash.list.diff.concat <- bla$name[bla$concatdiff==TRUE]

#elimination of these
restrictM <- is.element(database$HASH,Hash.list.diff.concat)

temp <- database[restrictM,] 

vec_eff <- table(database$YEAR[restrictM])
reasons$M[match(names(vec_eff), reasons$YEAR)] <- vec_eff

database.1 <- database[!restrictM,] #exclusion
sum(reasons$M,na.rm = TRUE)
length(database.1[,1]) #1323413(-6925)


#### N: true duplicates ####
##treatment of cases with equal "CONCAT" variable values

#we separate the DB into the OK and remaining problematic

database.1.ok <- database.1[!is.element(database.1$HASH,bla$name),]
length(database.1.ok[,1]) #[1] 1317141
database.1.nonok <- database.1[is.element(database.1$HASH,bla$name),]
length(database.1.nonok[,1]) #[1] 6272

# treating the OK
bla.diff <- bla[bla$concatdiff==FALSE,]
table(bla.diff$n>2)
table(database$UNIT_OF_MEASUREMENT_2[is.element(database$HASH,bla.diff$name)])
table(database$IMIS_SUBSTANCE_CODE[is.element(database$HASH,bla.diff$name)])
#0040 0430 0520 0685 0720 0731 1591 1631 1720 1790 2460 2590 9010 9015 9020 C141 S103 T177 
#   2    2   14    2    2    2   24   10    2    2    6   22 6084   50   30   12    4    2
true.duplicates <- database[is.element(database$HASH,bla.diff$name),]

# majority is 9010 (e.g. duplicates of "M" and "M.from.Perc" cases) 
# only 9010 treated, remaining cases are deleted
database.1.nonok.9010 <- database.1.nonok[database.1.nonok$IMIS_SUBSTANCE_CODE=="9010",]
length(database.1.nonok.9010[,1]) #[1]  6084
table(database.1.nonok.9010$UNIT_OF_MEASUREMENT_2)
#              %           M M.from.Perc 
#437        1177        2822        1648 
temp <- database.1.nonok.9010[database.1.nonok.9010$UNIT_OF_MEASUREMENT_2=='M.from.Perc',]
#data are sorted according to HASH and one out of 2 records is retained
database.1.nonok.9010 <- database.1.nonok.9010[order(database.1.nonok.9010$HASH),]
database.1.nonok.9010 <- database.1.nonok.9010[seq(from=1,to=6083,by=2),]
length(database.1.nonok.9010[,1]) #[1]  3042

# binding of two subsets of database.1
database.final <- rbind(database.1.ok,database.1.nonok.9010)
nrow(database.1) - nrow(database.final)
length(database.final[,1]) #[1] 1320183 (-3230)

[1] "V1"                    "AIR_VOLUME_SAMPLED"    "BLANK_USED"           
 [4] "CITY"                  "DATE_REPORTED"         "DATE_SAMPLED"         
 [7] "EIGHT_HOUR_TWA_CALC"   "ESTABLISHMENT_NAME"    "FIELD_NUMBER"         
[10] "IMIS_SUBSTANCE_CODE"   "INSPECTION_NUMBER"     "INSTRUMENT_TYPE"      
[13] "LAB_NUMBER"            "NAICS_CODE"            "OFFICE_ID"            
[16] "QUALIFIER"             "SAMPLE_RESULT"         "SAMPLE_TYPE"          
[19] "SAMPLE_WEIGHT"         "SAMPLING_NUMBER"       "SIC_CODE"             
[22] "STATE"                 "SUBSTANCE"             "TIME_SAMPLED"         
[25] "UNIT_OF_MEASUREMENT"   "YEAR"                  "ZIP_CODE"             
[28] "CENSORED"              "SAMPLE_RESULT_2"       "UNIT_OF_MEASUREMENT_2"
[31] "QUALIFIER_2"           "SAMPLE_WEIGHT_2"       "SAMPLE_RESULT_3"      
[34] "INSTRUMENT_TYPE_2"

[1] 6162

[1] 12342

[1] 1
[1] 2
[1] 3
[1] 4
[1] 5
[1] 6
[1] 7
[1] 8
[1] 9
[1] 10
[1] 11
[1] 12
[1] 13
[1] 14
[1] 15
[1] 16
[1] 17
[1] 18
[1] 19
[1] 20
[1] 21
[1] 22
[1] 23
[1] 24
[1] 25
[1] 26
[1] 27
[1] 28
[1] 29
[1] 30
[1] 31
[1] 32
[1] 33
[1] 34
[1] 35
[1] 36
[1] 37
[1] 38
[1] 39
[1] 40
[1] 41
[1] 42
[1] 43
[1] 44
[1] 45
[1] 46
[1] 47
[1] 48
[1] 49
[1] 50
[1] 51
[1] 52
[1] 53
[1] 54
[1] 55
[1] 56
[1] 57
[1] 58
[1] 59
[1] 60
[1] 61
[1] 62
[1] 63
[1] 64
[1] 65
[1] 66
[1] 67
[1] 68
[1] 69
[1] 70
[1] 71
[1] 72
[1] 73
[1] 74
[1] 75
[1] 76
[1] 77
[1] 78
[1] 79
[1] 80
[1] 81
[1] 82
[1] 83
[1] 84
[1] 85
[1] 86
[1] 87
[1] 88
[1] 89
[1] 90
[1] 91
[1] 92
[1] 93
[1] 94
[1] 95
[1] 96
[1] 97
[1] 98
[1] 99
[1] 100
[1] 101
[1] 102
[1] 103
[1] 104
[1] 105
[1] 106
[1] 107
[1] 108
[1] 109
[1] 110
[1] 111
[1] 112
[1] 113
[1] 114
[1] 115
[1] 116
[1] 117
[1] 118
[1] 119
[1] 120
[1] 121
[1] 122
[1] 123
[1] 124
[1] 125
[1] 126
[1] 127
[1] 128
[1] 129
[1] 130
[1] 131
[1] 132
[1] 133
[1] 134
[1] 135
[1] 136
[1] 137
[1] 138
[1] 


FALSE  TRUE 
 2870  3292 

...

[1] 1


          %           M M.from.Perc 
       1177        2821        1648 

[1] 1

[1] 2698

[1] 1

In [39]:
#### finalizing database ####
#finalizing variable class
str(database.final)

database.final$LAB_NUMBER <- as.character(database.final$LAB_NUMBER)
database.final$OFFICE_ID <- as.character(database.final$OFFICE_ID)
database.final$SIC_CODE <- as.character(database.final$SIC_CODE)
database.final$STATE <- as.character(database.final$STATE)
database.final$ZIP_CODE <- as.character(database.final$ZIP_CODE)

#final variable names
#CEHD <- read.csv("C:\\Users\\phisar\\Documents\\Data_CEHD\\CEHD.csv",sep=",", header=T)
#dim(CEHD) #[1] 1037395      23
#names(CEHD)
#[1] "inspection_number"     "establishment_name"    "city"                  "state"                 "zip_code"             
#[6] "sic_code"              "naics_code"            "sampling_number"       "office_id"             "date_sampled"         
#[11] "date_reported"         "field_number"          "time_sampled"          "imis_substance_code"   "substance"            
#[16] "unit_of_measurement_N" "sample_weight_N"       "air_volume_sampled_N"  "sample_result_N"       "instrument_type_N"    
#[21] "hash"                  "data_source"           "position_in_source"   

names(database.final)
#[1] "AIR_VOLUME_SAMPLED"    "BLANK_USED"            "CITY"                  "DATE_REPORTED"         "DATE_SAMPLED"         
#[6] "EIGHT_HOUR_TWA_CALC"   "ESTABLISHMENT_NAME"    "FIELD_NUMBER"          "IMIS_SUBSTANCE_CODE"   "INSPECTION_NUMBER"    
#[11] "INSTRUMENT_TYPE"       "LAB_NUMBER"            "NAICS_CODE"            "OFFICE_ID"             "QUALIFIER"            
#[16] "SAMPLE_RESULT"         "SAMPLE_TYPE"           "SAMPLE_WEIGHT"         "SAMPLING_NUMBER"       "SIC_CODE"             
#[21] "STATE"                 "SUBSTANCE"             "TIME_SAMPLED"          "UNIT_OF_MEASUREMENT"   "ZIP_CODE"             
#[26] "YEAR"                  "CENSORED"              "SAMPLE_RESULT_2"       "UNIT_OF_MEASUREMENT_2" "QUALIFIER_2"          
#[31] "SAMPLE_WEIGHT_2"       "SAMPLE_RESULT_3"       "INSTRUMENT_TYPE_2"     "HASH"                  "CONCAT"    

names(database.final)[1] <-'air_volume_sampled_N'
names(database.final)[3] <-'city'
names(database.final)[4] <-'date_reported'
names(database.final)[5] <-'date_sampled'
names(database.final)[7] <-'establishment_name'
names(database.final)[8] <-'field_number'
names(database.final)[9] <-'imis_substance_code'
names(database.final)[10] <-'inspection_number'
names(database.final)[12] <-'lab_number'
names(database.final)[13] <-'naics_code'
names(database.final)[14] <-'office_id'
names(database.final)[19] <-'sampling_number'
names(database.final)[20] <-'sic_code'
names(database.final)[21] <-'state'
names(database.final)[22] <-'substance'
names(database.final)[23] <-'time_sampled'
names(database.final)[25] <-'zip_code'
names(database.final)[26] <-'year'
names(database.final)[29] <-'unit_of_measurement_N'
names(database.final)[31] <-'sample_weight_N'
names(database.final)[32] <-'sample_result_N'
names(database.final)[33] <-'instrument_type_N'
names(database.final)[34] <-'hash'

# selection and ordering of final variables
database.final.1 <- database.final[,-c(2,6,11,15,16,17,18,24,27,28,30,35),drop=FALSE]
names(database.final.1)
#[1] "air_volume_sampled_N"  "city"                  "date_reported"         "date_sampled"          "establishment_name"   
#[6] "field_number"          "imis_substance_code"   "inspection_number"     "lab_number"            "naics_code"           
#[11] "office_id"             "sampling_number"       "sic_code"              "state"                 "substance"            
#[16] "time_sampled"          "zip_code"              "year"                  "unit_of_measurement_N" "sample_weight_N"      
#[21] "sample_result_N"       "instrument_type_N"     "hash"  
database.final.2 <- database.final.1[,c(8,5,2,14,17,13,10,12,11,9,4,3,6,18,7,15,22,20,16,1,21,19,23)]
dim(database.final.2)
# [1] 1320183      23

subst <- data.frame(table(database.final.2$imis_substance_code))
names(subst) <-c('code','n')
subst <- subst[order(-subst$n),]
nrow(subst) #[1] 257

# length(unique(database.final.2$hash))

# dim(cehd) #[1] 2395071      26
# # [1] 1320183      23

# length(cehd$instrument_type_N[cehd$instrument_type_N=="raw was NA"])

Classes 'data.table' and 'data.frame':	1296123 obs. of  36 variables:
 $ V1                   : int  4 7 9 85 118 142 143 144 145 146 ...
 $ AIR_VOLUME_SAMPLED   : num  74 74 74 0.783 3 5.4 3.3 5.8 6 5.85 ...
 $ BLANK_USED           : Factor w/ 2 levels "Y","N": 2 2 2 2 2 2 2 2 2 2 ...
 $ CITY                 : chr  "East Farmingdale" "East Farmingdale" "East Farmingdale" "Passaic" ...
 $ DATE_REPORTED        : Date, format: "1984-04-27" "1984-04-27" ...
 $ DATE_SAMPLED         : Date, format: "1984-04-04" "1984-04-04" ...
 $ EIGHT_HOUR_TWA_CALC  : Factor w/ 2 levels "Y","N": NA NA NA NA NA NA NA NA NA NA ...
 $ ESTABLISHMENT_NAME   : chr  "MID ISLAND NON FERROUS FOUNDRY" "MID ISLAND NON FERROUS FOUNDRY" "MID ISLAND NON FERROUS FOUNDRY" "PANTASOTE INC" ...
 $ FIELD_NUMBER         : chr  "2LI84-49" "2LI84-49" "2LI84-49" "K167" ...
 $ IMIS_SUBSTANCE_CODE  : chr  "1591" "1840" "0731" "2580" ...
 $ INSPECTION_NUMBER    : chr  "111252" "111252" "111252" "113951" ...
 $ INSTRUMENT_TYPE      

[1] "V1"                    "AIR_VOLUME_SAMPLED"    "BLANK_USED"           
 [4] "CITY"                  "DATE_REPORTED"         "DATE_SAMPLED"         
 [7] "EIGHT_HOUR_TWA_CALC"   "ESTABLISHMENT_NAME"    "FIELD_NUMBER"         
[10] "IMIS_SUBSTANCE_CODE"   "INSPECTION_NUMBER"     "INSTRUMENT_TYPE"      
[13] "LAB_NUMBER"            "NAICS_CODE"            "OFFICE_ID"            
[16] "QUALIFIER"             "SAMPLE_RESULT"         "SAMPLE_TYPE"          
[19] "SAMPLE_WEIGHT"         "SAMPLING_NUMBER"       "SIC_CODE"             
[22] "STATE"                 "SUBSTANCE"             "TIME_SAMPLED"         
[25] "UNIT_OF_MEASUREMENT"   "YEAR"                  "ZIP_CODE"             
[28] "CENSORED"              "SAMPLE_RESULT_2"       "UNIT_OF_MEASUREMENT_2"
[31] "QUALIFIER_2"           "SAMPLE_WEIGHT_2"       "SAMPLE_RESULT_3"      
[34] "INSTRUMENT_TYPE_2"     "HASH"                  "CONCAT"

[1] "air_volume_sampled_N"  "city"                  "date_reported"        
 [4] "date_sampled"          "establishment_name"    "field_number"         
 [7] "imis_substance_code"   "inspection_number"     "lab_number"           
[10] "naics_code"            "office_id"             "sampling_number"      
[13] "sic_code"              "state"                 "substance"            
[16] "time_sampled"          "zip_code"              "year"                 
[19] "unit_of_measurement_N" "sample_weight_N"       "sample_result_N"      
[22] "instrument_type_N"     "hash"                  "CONCAT"

[1] 1296123      23

[1] 229775

In [40]:
write_parquet(database, 'database.parquet')
write_parquet(database.final.1, 'dbf1.parquet')
write_parquet(database.final.2, 'dbf2.parquet')